In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
pd.options.display.max_rows = 6

# groupby를 활용하여 그룹별 전처리

In [2]:
tips = sns.load_dataset('tips')

## 기본

### get_group
 - 특정 그룹의 결과만 가져오도록 한다.

In [3]:
tips_grouped = tips.groupby('sex')

In [10]:
tips_grouped.get_group('Male')

,total_bill,tip,sex,smoker,day,time,size
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


### 0. 기본 함수 적용

In [37]:
tips.groupby('sex').head(2)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
4,24.59,3.61,Female,No,Sun,Dinner,4


In [38]:
tips.groupby('sex').sum()

,total_bill,tip,size
sex,,,
Male,3256.82,485.07,413
Female,1570.95,246.51,214


In [5]:
tips.groupby('sex').nunique()

,total_bill,tip,sex,smoker,day,time,size
sex,,,,,,,
Male,151,90,1,2,4,2,6
Female,84,49,1,2,4,2,6


### 1. count & size

count : NULL 포함하지 않음 / size : NULL 포함

참고 : count는 컬럼 지정하지 않으면 모든 컬럼을 수행

In [7]:
tips.groupby('sex').size()

sex
Male      157
Female     87
dtype: int64

In [8]:
tips.groupby('sex').count()

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Male,157,157,157,157,157,157
Female,87,87,87,87,87,87


#### as_index : SQL 스타일로 표현

In [9]:
tips.groupby('sex', as_index=False).count()

,sex,total_bill,tip,smoker,day,time,size
0,Male,157,157,157,157,157,157
1,Female,87,87,87,87,87,87


In [11]:
tips.groupby('sex')['size'].count()

sex
Male      157
Female     87
Name: size, dtype: int64

#### sort option

계산결과를 컬럼값 기준으로 정리 / SQL에서 groupby + order by 와 같음

In [10]:
tips.groupby('sex', as_index=False)['size'].count()

,sex,size
0,Male,157
1,Female,87


In [12]:
tips.groupby('sex', sort=False)['size'].count()

sex
Female     87
Male      157
Name: size, dtype: int64

### 2. agg : 각 그룹마다 함수를 계산
    -  하나의 value를 return

In [15]:
def min_max(group):
    return group.max() - group.min()

In [16]:
tips.groupby('sex').agg(min_max)

,total_bill,tip,size
sex,,,
Male,43.56,9.0,5
Female,41.23,5.5,5


#### 각 컬럼별로 다른 함수로 계산 가능

In [6]:
tips.groupby('sex').agg({'sex':len,'tip':np.mean, 'size':np.max})

,sex,tip,size
sex,,,
Male,157,3.089618,6
Female,87,2.833448,6


### 3. filter : 그룹과 연관된 조건식으로 전체 ROW 필터링

In [13]:
tips.groupby('sex')['size'].mean()

sex
Male      2.630573
Female    2.459770
Name: size, dtype: float64

In [14]:
# female은 size의 평균이 2.5보다 작기 때문에 male만 추출되는 결과
test = tips.groupby('sex').filter(lambda group: group['size'].mean() >=2.5)
test

,total_bill,tip,sex,smoker,day,time,size
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


### 4. transform : apply처럼 모든 값에 대하여 계산
    -  해당 value가 속하는 그룹의 결과(mean, sum 등)를 활용할 수 있다는 것이 다름

In [8]:
tips[:2]

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3


In [9]:
def zscore(group):
    return (group - group.mean()) / group.std()

In [10]:
# 남자는 남자의 평균과 표준편차, 여자는 여자의 평균과 표준편차를 활용하여 계산한다.
tips.groupby('sex').tip.transform(zscore)

0     -1.572623
1     -0.960054
2      0.275590
         ...   
241   -0.731728
242   -0.899615
243    0.143642
Name: tip, Length: 244, dtype: float64

In [11]:
# 그룹별로 잘 계산이 되는 것 검증
by_total = round((tips.tip[0] - tips.tip.mean())/tips.tip.std(),5)
by_female = round((tips.tip[0] - tips.groupby('sex').tip.mean()['Female'])/tips.groupby('sex').tip.std()['Female'],5)
by_male = round((tips.tip[0] - tips.groupby('sex').tip.mean()['Male'])/tips.groupby('sex').tip.std()['Male'],5)
print('0번 row 결과 확인')
print("전체로 계산 : ", by_total ) 
print("여자 그룹으로 계산 : ", by_female) # transform 결과와 일치
print('\n1번 row 결과 확인')
print("전체로 계산 : ", by_total)
print("남자 그룹으로 계산 : ", by_male) # transform 결과와 일치

0번 row 결과 확인
전체로 계산 :  -1.43699
여자 그룹으로 계산 :  -1.57262

1번 row 결과 확인
전체로 계산 :  -1.43699
남자 그룹으로 계산 :  -1.39656


## SQL의 WINDOW FUNCTION 기능

> .groupby가 SQL의 PARTITION BY 역할을 수행

### 1. cumsum : 누적합

SQL의 SUM(컬럼) OVER(PARTITION BY 컬럼 ORDER BY 컬럼 )과 같다.

In [97]:
tips = sns.load_dataset('tips')

In [98]:
tips_cumsum = tips.assign(cum_sum = tips.sort_values(by=['tip']).groupby(['sex'])['tip'].cumsum())

In [99]:
tips_cumsum.sort_values(by=['sex','tip'])

,total_bill,tip,sex,smoker,day,time,size,cum_sum
236,12.60,1.00,Male,Yes,Sat,Dinner,2,1.00
237,32.83,1.17,Male,Yes,Sat,Dinner,2,2.17
75,10.51,1.25,Male,No,Sat,Dinner,2,4.67
...,...,...,...,...,...,...,...,...
85,34.83,5.17,Female,No,Thur,Lunch,4,234.81
52,34.81,5.20,Female,No,Sun,Dinner,4,240.01
214,28.17,6.50,Female,Yes,Sat,Dinner,3,246.51


### 2. rank

method option에 따라 SQL의 RANK, DENSE_RANK, ROW_NUMBER 선택 가능

ROW_NUMBER() OVER (PARTITION BY 컬럼 ORDER BY 컬럼)

참고 : https://www.w3resource.com/pandas/dataframe/dataframe-rank.php

In [41]:
# method: first - ROW_NUMBER
tips_ranked = tips.assign(row_number=tips.groupby(['sex'])['size'].rank(method='first', ascending=False)) \
    .query('row_number <= 3').sort_values(by=['sex','row_number'])

In [42]:
tips_ranked

,total_bill,tip,sex,smoker,day,time,size,row_number
141,34.30,6.70,Male,No,Thur,Lunch,6,1.0
156,48.17,5.00,Male,No,Sun,Dinner,6,2.0
142,41.19,5.00,Male,No,Thur,Lunch,5,3.0
125,29.80,4.20,Female,No,Thur,Lunch,6,1.0
143,27.05,5.00,Female,No,Thur,Lunch,6,2.0
155,29.85,5.14,Female,No,Sun,Dinner,5,3.0
